In [117]:
import pandas as pd
import os

In [118]:
print(os.getcwd())
os.chdir('../raw_data')
print(os.getcwd())

/Users/kumo/hec/courses/data_challenge_uw/Data-Challenge-URW-HEC/raw_data
/Users/kumo/hec/courses/data_challenge_uw/Data-Challenge-URW-HEC/raw_data


In [119]:
data = pd.read_csv('cleaned_data.csv')
data.head(5)

,Unnamed: 0,external_mall_id,block_id,average_dwell_time,median_dwell_time,store_id,store_visits_monthly_average,store_window_flow_monthly_average,store_visits_daily_average,store_window_flow_daily_average,...,store_branch_level_1,store_branch_level_2,store_branch_level_3,store_sales_r12m,store_rent_r12m,store_area,service_charges_r12m,marketing_costs_r12m,real_estate_taxes_r12m,maintenance_works_r12m
0,0,18,11273,249.520051,180.0,9257.0,9810.25,472952.50,509.62,24568.96,...,Jewellery,Jewellery & Watches,Modern Jewellery & Watches,886377.00,NaN,104.0,25034.0,0.0,11957.0,6337.0
1,1,18,11271,190.005359,120.0,9443.0,4891.25,362107.00,254.09,18810.75,...,Fashion apparel,Family Fashion,Family Fashion,20883319.00,NaN,203.0,33465.0,0.0,11957.0,6337.0
2,2,17,4319,1325.216860,780.0,5697.0,1981.23,280719.08,70.37,9970.90,...,Services,Imaterial services,Travel Agency,2339819.57,140492.0,72.0,18000.0,980.0,11957.0,6337.0
3,3,36,1201,621.422549,240.0,7179.0,29753.50,751216.75,1226.95,30978.01,...,Food & Beverage Services,Sweet moments,Confectionery,1129538.00,89064.0,107.0,80035.0,5038.0,11957.0,24846.0
4,4,36,1212,217.833860,120.0,7288.0,6082.50,357884.00,253.44,14911.83,...,Fashion apparel,Men's Fashion,Men's Fashion,340091.00,2181010.0,208.0,69226.0,0.0,11957.0,6337.0


In [120]:
data.columns

Index(['Unnamed: 0', 'external_mall_id', 'block_id', 'average_dwell_time',
       'median_dwell_time', 'store_id', 'store_visits_monthly_average',
       'store_window_flow_monthly_average', 'store_visits_daily_average',
       'store_window_flow_daily_average', 'store_total_visits',
       'store_total_window_flow', 'days_with_traffic_data', 'mall_name',
       'mall_country', 'mall_id', 'total_mall_area', 'store_name',
       'store_year_opened', 'store_year_closed', 'store_branch_level_1',
       'store_branch_level_2', 'store_branch_level_3', 'store_sales_r12m',
       'store_rent_r12m', 'store_area', 'service_charges_r12m',
       'marketing_costs_r12m', 'real_estate_taxes_r12m',
       'maintenance_works_r12m'],
      dtype='object')

# cleaning

## cost

In [121]:
cost_cols = ['service_charges_r12m', 'marketing_costs_r12m', 'real_estate_taxes_r12m', 'maintenance_works_r12m']
data[cost_cols].min(axis=0)
# negative values should not exist, we determine to use 0 to replace them.

service_charges_r12m     -86042.0
marketing_costs_r12m     -41943.0
real_estate_taxes_r12m   -16695.0
maintenance_works_r12m        0.0
dtype: float64

In [122]:
print(f"percentage(%) of negative values\n{((data[cost_cols]<0).sum(axis=0)/data.shape[0])*100}")

percentage(%) of negative values
service_charges_r12m      1.673066
marketing_costs_r12m      0.246846
real_estate_taxes_r12m    0.054855
maintenance_works_r12m    0.000000
dtype: float64


In [123]:
data[cost_cols].sum(axis=1)

0        43328.0
1        51759.0
2        37274.0
3       121876.0
4        87520.0
          ...   
3641     90516.0
3642     61794.0
3643     19760.0
3644     49073.0
3645     19781.0
Length: 3646, dtype: float64

In [124]:
# replace them with zero
data[cost_cols] = data[cost_cols].applymap(lambda x:max(x, 0))
data['total_costs_r12m'] = data[cost_cols].sum(axis=1)

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/2867994817.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[cost_cols] = data[cost_cols].applymap(lambda x:max(x, 0))


## visitor

In [146]:
data['store_visits_monthly_average'].describe()

count      3646.000000
mean      21605.634290
std       34102.897853
min           0.000000
25%        4737.047500
50%       10198.545000
75%       23670.922500
max      444736.540000
Name: store_visits_monthly_average, dtype: float64

there're zero observations

# metrics

In [ ]:
identity = ['store_id', 'block_id', 'mall_id', 'external_mall_id']
metrics = data[identity]

make them based on month

## traffic

### traffic cost efficiency

In [126]:
# traffic cost efficiency
metrics['tfc_cst_efc'] = data['store_window_flow_monthly_average']/(data['total_costs_r12m']/12) 

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/2077493001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['tfc_cst_efc'] = data['store_window_flow_monthly_average']/(data['total_costs_r12m']/12)


### visit rate

In [127]:
# visit rate
# there are monthly, daily and total 
metrics['monthly_visit/window'] = data['store_visits_monthly_average'] / data['store_window_flow_monthly_average']
metrics['daily_visit/window'] = data['store_visits_daily_average'] / data['store_window_flow_daily_average']
metrics['total_visit/window'] = data['store_total_visits'] / data['store_total_window_flow']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1795006489.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['monthly_visit/window'] = data['store_visits_monthly_average'] / data['store_window_flow_monthly_average']
/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1795006489.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['daily_visit/window'] = data['store_visits_daily_average'] / data['store_window_flow_daily_average']
/var/folders/z9/zf5v62yj5_75v0dw

In [128]:
metrics.head(10)
# since they are almost the same, we only keep one
metrics.drop(['daily_visit/window', 'total_visit/window'], axis=1, inplace=True)
metrics.rename(columns={'monthly_visit/window':'visit_rate'}, inplace=True)

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1917769999.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics.drop(['daily_visit/window', 'total_visit/window'], axis=1, inplace=True)
/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1917769999.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics.rename(columns={'monthly_visit/window':'visit_rate'}, inplace=True)


## Sales

### revenue per visitor

In [129]:
# revenue per visit
# time unit, month
metrics['RPV'] = (data['store_sales_r12m']/12)/data['store_visits_monthly_average']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/2476251328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['RPV'] = (data['store_sales_r12m']/12)/data['store_visits_monthly_average']


### revenue per m2

In [130]:
# revenue per m2
# RPM 
metrics['RPM'] = (data['store_sales_r12m']/12)/data['store_area']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/3968750639.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['RPM'] = (data['store_sales_r12m']/12)/data['store_area']


### revenue efficiency

In [131]:
# Rvenue efficiency
metrics['RE'] = data['store_sales_r12m']/data['total_costs_r12m']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/3601272623.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['RE'] = data['store_sales_r12m']/data['total_costs_r12m']


In [132]:
metrics['RE'].sort_values()

3292    0.000000
121     0.000000
3171    0.000000
3256    0.000000
2174    0.001609
          ...   
3628         NaN
3639         NaN
3640         NaN
3641         NaN
3643         NaN
Name: RE, Length: 3646, dtype: float64

In [133]:
data.columns

Index(['Unnamed: 0', 'external_mall_id', 'block_id', 'average_dwell_time',
       'median_dwell_time', 'store_id', 'store_visits_monthly_average',
       'store_window_flow_monthly_average', 'store_visits_daily_average',
       'store_window_flow_daily_average', 'store_total_visits',
       'store_total_window_flow', 'days_with_traffic_data', 'mall_name',
       'mall_country', 'mall_id', 'total_mall_area', 'store_name',
       'store_year_opened', 'store_year_closed', 'store_branch_level_1',
       'store_branch_level_2', 'store_branch_level_3', 'store_sales_r12m',
       'store_rent_r12m', 'store_area', 'service_charges_r12m',
       'marketing_costs_r12m', 'real_estate_taxes_r12m',
       'maintenance_works_r12m', 'total_costs_r12m'],
      dtype='object')

In [134]:
data.loc[1642, ['service_charges_r12m',
       'marketing_costs_r12m', 'real_estate_taxes_r12m',
       'maintenance_works_r12m']]

service_charges_r12m          0.0
marketing_costs_r12m      14146.0
real_estate_taxes_r12m      884.0
maintenance_works_r12m     6337.0
Name: 1642, dtype: object

## cost

### cost per m2

In [ ]:
# cost per m2
metrics['CPM'] = (data['total_costs_r12m']/12)/data['store_area']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/4174579882.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['CPM'] = (data['total_costs_r12m']/12)/data['store_area']
/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/4174579882.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['PM'] = (data['store_sales_r12m']-data['total_costs_r12m'])/data['store_sales_r12m']


### profit margin

In [ ]:
# profit margin 
metrics['PM'] = (data['store_sales_r12m']-data['total_costs_r12m'])/data['store_sales_r12m']

## engagement

In [136]:
# using median dwell in case outlier effect
metrics['Space_u'] = data['median_dwell_time']/data['store_area']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/2825203873.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['Space_u'] = data['median_dwell_time']/data['store_area']


In [137]:
metrics.head(5)

,store_id,block_id,mall_id,external_mall_id,tfc_cst_efc,visit_rate,RPV,RPM,RE,CPM,PM,Space_u
0,9257.0,11273,54.0,18,130.987583,0.020743,7.529344,710.237981,20.457372,34.717949,0.951118,1.730769
1,9443.0,11271,54.0,18,83.952240,0.013508,355.793833,8572.791051,403.472227,21.247537,0.997522,0.591133
2,5697.0,4319,15.0,17,90.374764,0.007058,98.416117,2708.124502,62.773504,43.141204,0.984070,10.833333
3,7179.0,1201,35.0,36,73.965350,0.039607,3.163600,879.702492,9.267928,94.919003,0.892101,2.242991
4,7288.0,1212,35.0,36,49.070018,0.016996,4.659419,136.254407,3.885866,35.064103,0.742657,0.576923


In [138]:
metrics.isna().sum(axis=0)

store_id              0
block_id              0
mall_id               0
external_mall_id      0
tfc_cst_efc           0
visit_rate            0
RPV                 884
RPM                 884
RE                  884
CPM                   0
PM                  884
Space_u               0
dtype: int64

# comprehensive metric

In [142]:
metrics.columns

Index(['store_id', 'block_id', 'mall_id', 'external_mall_id', 'tfc_cst_efc',
       'visit_rate', 'RPV', 'RPM', 'RE', 'CPM', 'PM', 'Space_u'],
      dtype='object')

In [144]:
m_cols = ['tfc_cst_efc','visit_rate', 'RPV', 'RPM', 'RE', 'CPM', 'PM', 'Space_u']
metrics[m_cols].describe()

/opt/anaconda3/envs/foo-env/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/opt/anaconda3/envs/foo-env/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/opt/anaconda3/envs/foo-env/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/opt/anaconda3/envs/foo-env/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/opt/anaconda3/envs/foo-env/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,tfc_cst_efc,visit_rate,RPV,RPM,RE,CPM,PM,Space_u
count,3646.000000,3646.000000,2762.000000,2762.000000,2762.000000,3646.000000,2762.000000,3646.000000
mean,187.088252,0.038444,inf,inf,136.044925,inf,-inf,inf
std,233.682046,0.056209,NaN,NaN,539.972247,NaN,NaN,NaN
min,0.582914,0.000000,0.000000,0.000000,0.000000,0.085344,-inf,0.023414
25%,61.609930,0.009570,2.876548,185.289422,7.894056,11.049990,0.873322,0.821918
50%,123.524933,0.019167,9.630544,659.229132,22.572572,26.985722,0.955698,1.846154
75%,234.780551,0.043041,32.357175,2822.363192,68.018799,72.616789,0.985298,4.766553
max,5204.653960,0.790122,inf,inf,9772.530218,inf,0.999898,inf
